In [1]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Quantum Generative Adversarial Networks with Cirq + TensorFlow
==============================================================

*Author: PennyLane dev team. Last updated: 15 Jan 2021.*

This demo constructs a Quantum Generative Adversarial Network (QGAN)
([Lloyd and Weedbrook
(2018)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.121.040502),
[Dallaire-Demers and Killoran
(2018)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.012324))
using two subcircuits, a *generator* and a *discriminator*. The
generator attempts to generate synthetic quantum data to match a pattern
of "real" data, while the discriminator tries to discern real data from
fake data (see image below). The gradient of the discriminator’s output
provides a training signal for the generator to improve its fake
generated data.

|

![](../demonstrations/QGAN/qgan.png){width="75%"}

|


Using Cirq + TensorFlow
=======================

PennyLane allows us to mix and match quantum devices and classical
machine learning software. For this demo, we will link together Google's
[Cirq](https://cirq.readthedocs.io/en/stable/) and
[TensorFlow](https://www.tensorflow.org/) libraries.

We begin by importing PennyLane, NumPy, and TensorFlow.


In [3]:
!pip install pennylane pennylane-cirq
import pennylane as qml
import numpy as np
import tensorflow as tf

     |████████████████████████████████| 514 kB 4.9 MB/s 
     |████████████████████████████████| 380 kB 9.4 MB/s 
     |████████████████████████████████| 1.5 MB 11.0 MB/s 
     |████████████████████████████████| 1.3 MB 28.2 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3


We also declare a 3-qubit simulator device running in Cirq.


In [2]:
import pennylane as qml
dev = qml.device('cirq.simulator', wires=3)

Generator and Discriminator
===========================

In classical GANs, the starting point is to draw samples either from
some "real data" distribution, or from the generator, and feed them to
the discriminator. In this QGAN example, we will use a quantum circuit
to generate the real data.

For this simple example, our real data will be a qubit that has been
rotated (from the starting state $\left|0\right\rangle$) to some
arbitrary, but fixed, state.


In [3]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.Rot(*angles, wires=0)

For the generator and discriminator, we will choose the same basic
circuit structure, but acting on different wires.

Both the real data circuit and the generator will output on wire 0,
which will be connected as an input to the discriminator. Wire 1 is
provided as a workspace for the generator, while the discriminator’s
output will be on wire 2.


In [4]:
def generator(w, **kwargs):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=1)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RX(w[6], wires=0)
    qml.RY(w[7], wires=0)
    qml.RZ(w[8], wires=0)


def discriminator(w):
    qml.Hadamard(wires=0)
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=2)
    qml.RY(w[2], wires=0)
    qml.RY(w[3], wires=2)
    qml.RZ(w[4], wires=0)
    qml.RZ(w[5], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

We create two QNodes. One where the real data source is wired up to the
discriminator, and one where the generator is connected to the
discriminator. In order to pass TensorFlow Variables into the quantum
circuits, we specify the `"tf"` interface.


In [5]:
@qml.qnode(dev, interface="tf")
def real_disc_circuit(phi, theta, omega, disc_weights):
    real([phi, theta, omega])
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))


@qml.qnode(dev, interface="tf")
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights)
    return qml.expval(qml.PauliZ(2))

QGAN cost functions
===================

There are two cost functions of interest, corresponding to the two
stages of QGAN training. These cost functions are built from two pieces:
the first piece is the probability that the discriminator correctly
classifies real data as real. The second piece is the probability that
the discriminator classifies fake data (i.e., a state prepared by the
generator) as real.

The discriminator is trained to maximize the probability of correctly
classifying real data, while minimizing the probability of mistakenly
classifying fake data.

$$Cost_D = \mathrm{Pr}(real|\mathrm{fake}) - \mathrm{Pr}(real|\mathrm{real})$$

The generator is trained to maximize the probability that the
discriminator accepts fake data as real.

$$Cost_G = - \mathrm{Pr}(real|\mathrm{fake})$$


In [6]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(phi, theta, omega, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)

Training the QGAN
=================

We initialize the fixed angles of the "real data" circuit, as well as
the initial parameters for both generator and discriminator. These are
chosen so that the generator initially prepares a state on wire 0 that
is very close to the $\left| 1 \right\rangle$ state.


In [9]:
import numpy as np
import tensorflow as tf
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
init_disc_weights = np.random.normal(size=(9,))

gen_weights = tf.Variable(init_gen_weights)
disc_weights = tf.Variable(init_disc_weights)

We begin by creating the optimizer:


In [10]:
opt = tf.keras.optimizers.SGD(0.4)

In the first stage of training, we optimize the discriminator while
keeping the generator parameters fixed.


In [11]:
cost = lambda: disc_cost(disc_weights)

for step in range(50):
    opt.minimize(cost, disc_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.05727687478065491
Step 5: cost = -0.26348111033439636
Step 10: cost = -0.4273917004466057
Step 15: cost = -0.47261590510606766
Step 20: cost = -0.48406896367669106
Step 25: cost = -0.48946382384747267
Step 30: cost = -0.49281889386475086
Step 35: cost = -0.4949494309257716
Step 40: cost = -0.49627021909691393
Step 45: cost = -0.49707187968306243


At the discriminator’s optimum, the probability for the discriminator to
correctly classify the real data should be close to one.


In [12]:
print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())

Prob(real classified as real):  0.9985871425596997


For comparison, we check how the discriminator classifies the
generator’s (still unoptimized) fake data:


In [13]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.5011128038167953


In the adversarial game we now have to train the generator to better
fool the discriminator. For this demo, we only perform one stage of the
game. For more complex models, we would continue training the models in
an alternating fashion until we reach the optimum point of the
two-player adversarial game.


In [14]:
cost = lambda: gen_cost(gen_weights)

for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = -0.5833386033773422
Step 5: cost = -0.8915732949972153
Step 10: cost = -0.9784244522452354
Step 15: cost = -0.9946483590174466
Step 20: cost = -0.9984995491686277
Step 25: cost = -0.9995636216044659
Step 30: cost = -0.9998718172573717
Step 35: cost = -0.9999619696027366
Step 40: cost = -0.9999888275397097
Step 45: cost = -0.999996672290763


At the optimum of the generator, the probability for the discriminator
to be fooled should be close to 1.


In [15]:
print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

Prob(fake classified as real):  0.99999862746688


At the joint optimum the discriminator cost will be close to zero,
indicating that the discriminator assigns equal probability to both real
and generated data.


In [16]:
print("Discriminator cost: ", disc_cost(disc_weights).numpy())

Discriminator cost:  0.0014114849071802382


The generator has successfully learned how to simulate the real data
enough to fool the discriminator.

Let's conclude by comparing the states of the real data circuit and the
generator. We expect the generator to have learned to be in a state that
is very close to the one prepared in the real data circuit. An easy way
to access the state of the first qubit is through its [Bloch
sphere](https://en.wikipedia.org/wiki/Bloch_sphere) representation:


In [17]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.2169418   0.45048445 -0.86602525]
Generator Bloch vector: [-0.2840465   0.41893208 -0.86244407]


#This code is a part of PennyLane
© Copyright PennyLane, 2019.

This code is licensed under the Apache License, Version 
2.0. You may obtain a copy of this license in the 
LICENSE.txt file in the root directory of this source tree 
or at http://www.apache.org/licenses/LICENSE-2.0.


Program executed by Bhadale IT in Google Colab 
(https://www.bhadaleit.com).
For more details on the PennyLane code and tutorials 
visit https://pennylane.ai/qml/ website